In [ ]:
using QuasiCrystal
using LinearAlgebra, Plots


projection の方法
r に対して計算コストが O(r^4)
一方で、生成される点の数は O(r^2) である
こう考えると、点の生成だけで案外時間がかかってしまうのか。


In [ ]:
θ = π / 4
radius = 3

E_par = zeros(4, 2)
E_perp = zeros(4, 2)
for i in 1:4
    E_par[i, :] = [cos(θ * (i - 1)), sin(θ * (i - 1))]
    E_perp[i, :] = [cos(θ * (i - 1) + π / 4), sin(θ * (i - 1) + π / 4)]
end

window_size = 0.5

positions = Vector{Float64}[]

pos_all = Vector{Float64}[]
pos_par = Vector{Float64}[]
pos_perp = Vector{Float64}[]
n_max = ceil(Int, radius * 2.0)
for n1 in -n_max:n_max
    for n2 in -n_max:n_max
        for n3 in -n_max:n_max
            for n4 in -n_max:n_max
                lattice_point = float.([n1, n2, n3, n4])
                push!(pos_all, E_par' * lattice_point)

                if norm(pos_all[end]) > radius
                    continue
                end
                push!(pos_par, E_par' * lattice_point)
                push!(pos_perp, E_perp' * lattice_point)
                if norm(pos_perp[end]) <= window_size
                    push!(positions, pos_par[end])
                end
            end
        end
    end
end
θ_circle = range(0, 2π, length=100)

pos_all = reduce(hcat, pos_all)
pos_par = reduce(hcat, pos_par)
pos_perp = reduce(hcat, pos_perp)
positions = reduce(hcat, positions)
p1 = scatter(pos_all[1, :], pos_all[2, :], aspect_ratio=1, markersize=2, title="All lattice points", label="")
cx = radius .* cos.(θ_circle)
cy = radius .* sin.(θ_circle)
plot!(p1, cx, cy, color=:red, lw=2, label="radius")
p2 = scatter(pos_par[1, :], pos_par[2, :], aspect_ratio=1, markersize=2, title="Parallel space", label="")
cx = radius .* cos.(θ_circle)
cy = radius .* sin.(θ_circle)
plot!(p2, cx, cy, color=:red, lw=2, label="radius")
p3 = scatter(pos_perp[1, :], pos_perp[2, :], aspect_ratio=1, markersize=2, title="Perpendicular space", label="")
cx = window_size .* cos.(θ_circle)
cy = window_size .* sin.(θ_circle)
plot!(p3, cx, cy, color=:red, lw=2, label="Window")
p4 = scatter(positions[1, :], positions[2, :], aspect_ratio=1, markersize=2, title="Accepted points", label="")

pp = plot([p1, p2, p3, p4]..., layout=(2, 2), size=(800, 800))


In [ ]:
θ = π / 4
radius = 3
for radius in [3, 5, 7]
    E_par = zeros(4, 2)
    E_perp = zeros(4, 2)
    for i in 1:4
        E_par[i, :] = [cos(θ * (i - 1)), sin(θ * (i - 1))]
        E_perp[i, :] = [cos(θ * (i - 1) + π / 4), sin(θ * (i - 1) + π / 4)]
    end

    window_size = 0.5

    positions = Vector{Float64}[]

    pos_all = Vector{Float64}[]
    pos_par = Vector{Float64}[]
    pos_perp = Vector{Float64}[]
    n_max = ceil(Int, radius * 1.5)
    for n1 in -n_max:n_max
        for n2 in -n_max:n_max
            for n3 in -n_max:n_max
                for n4 in -n_max:n_max
                    lattice_point = float.([n1, n2, n3, n4])
                    push!(pos_all, E_par' * lattice_point)

                    if norm(pos_all[end]) > radius
                        continue
                    end
                    push!(pos_par, E_par' * lattice_point)
                    push!(pos_perp, E_perp' * lattice_point)
                    if norm(pos_perp[end]) <= window_size
                        push!(positions, pos_par[end])
                    end
                end
            end
        end
    end
    θ_circle = range(0, 2π, length=100)

    pos_all = reduce(hcat, pos_all)
    pos_par = reduce(hcat, pos_par)
    pos_perp = reduce(hcat, pos_perp)
    positions = reduce(hcat, positions)
    p1 = scatter(pos_all[1, :], pos_all[2, :], aspect_ratio=1, markersize=2, title="All lattice points", label="")
    cx = radius .* cos.(θ_circle)
    cy = radius .* sin.(θ_circle)
    plot!(p1, cx, cy, color=:red, lw=2, label="radius")
    p2 = scatter(pos_par[1, :], pos_par[2, :], aspect_ratio=1, markersize=2, title="Parallel space", label="")
    cx = radius .* cos.(θ_circle)
    cy = radius .* sin.(θ_circle)
    plot!(p2, cx, cy, color=:red, lw=2, label="radius")
    p3 = scatter(pos_perp[1, :], pos_perp[2, :], aspect_ratio=1, markersize=2, title="Perpendicular space", label="")
    cx = window_size .* cos.(θ_circle)
    cy = window_size .* sin.(θ_circle)
    plot!(p3, cx, cy, color=:red, lw=2, label="Window")
    p4 = scatter(positions[1, :], positions[2, :], aspect_ratio=1, markersize=2, title="Accepted points", label="")

    pp = plot([p1, p2, p3, p4]..., layout=(2, 2), size=(800, 800))
    display(pp)
end


substitution のほうが基本的に簡単かもしれないねぇ...  
計算コスト、bond の生成という両面において


ちょっと経過として koch snowflake
sierpinski gasket
dragon curve
というようなものを生成したい


In [ ]:
using LinearAlgebra, StaticArrays, Plots

# 線分(p1, p2)を4本の線分に分割する関数
function subdivide_koch(p1, p2)
    v = p2 - p1
    q1 = p1 + v / 3
    q3 = p1 + 2v / 3

    # 頂点q2：中央を60度回転させて山を作る
    # 回転行列（60度）
    rot60 = @SMatrix [cos(π / 3) -sin(π / 3); sin(π / 3) cos(π / 3)]
    q2 = q1 + rot60 * (v / 3)

    return [p1, q1, q2, q3, p2] # 新しい頂点順
end

# 1世代分を計算する
# points: 現在の頂点リスト
function next_generation(points)
    new_points = SVector{2,Float64}[]
    for i in 1:(length(points)-1)
        res = subdivide_koch(points[i], points[i+1])
        # 重複を避けるため最後以外を追加
        append!(new_points, res[1:end-1])
    end
    push!(new_points, points[end]) # 最後の最後だけ追加
    return new_points
end

# 初期状態（水平な線）
pts = [SVector(0.0, 0.0), SVector(1.0, 0.0)]

# 4世代回す
for g in 1:4
    pts = next_generation(pts)
end

# 描画
mat = reduce(hcat, pts)
plot(mat[1, :], mat[2, :], aspect_ratio=1, lw=1, label="Koch Gen 4")


In [ ]:
using LinearAlgebra, StaticArrays, Plots

abstract type AbstractTile{N,T} end
struct KochTile{N,T} <: AbstractTile{N,T}
    level::Int
    vertices::Vector{SVector{N,T}}
    center::SVector{N,T}
end
function initiate_koch_tile(; level=0)
    v1 = SVector{2,Float64}(0.0, 0.0)
    v2 = SVector{2,Float64}(1.0, 0.0)
    center = (v1 + v2) / 2
    KochTile{2,Float64}(level, [v1, v2], center)
end
function subdivide_koch(p1, p2)
    v = p2 - p1
    q1 = p1 + v / 3 # 1/3 point
    q3 = p1 + 2v / 3 # 2/3 point

    # rotated point q2
    rot60 = @SMatrix [cos(π / 3) -sin(π / 3); sin(π / 3) cos(π / 3)]
    q2 = q1 + rot60 * (v / 3)
    return [p1, q1, q2, q3, p2]
end

function next_generation(tile::KochTile{2,Float64})
    level = tile.level + 1
    new_vertices = SVector{2,Float64}[]
    old_vertices = tile.vertices
    for i in 1:(length(old_vertices)-1)
        res = subdivide_koch(old_vertices[i], old_vertices[i+1])
        append!(new_vertices, res[1:end-1])
    end
    push!(new_vertices, old_vertices[end])
    center = sum(new_vertices) / length(new_vertices)
    KochTile{2,Float64}(level, new_vertices, center)
end

tile = initiate_koch_tile(level=0)
for g in 1:6
    tile = next_generation(tile)
end
pts = tile.vertices

mat = reduce(hcat, pts)
plot(mat[1, :], mat[2, :], aspect_ratio=1, lw=1, label="Koch Gen 4")


In [ ]:
using LinearAlgebra, StaticArrays, Plots

abstract type AbstractTile{N,T} end
struct KochTile{N,T} <: AbstractTile{N,T}
    level::Int
    vertices::Vector{SVector{N,T}}
    center::SVector{N,T}
end
function initiate_koch_tile(; level=0)
    v1 = SVector{2,Float64}(0.0, 0.0)
    v2 = SVector{2,Float64}(1.0, 0.0)
    v3 = SVector{2,Float64}(0.5, √3 / 2)
    center = (v1 + v2 + v3) / 3
    KochTile{2,Float64}(level, [v1, v2, v3, v1], center)
end
function subdivide_koch(p1, p2)
    v = p2 - p1
    q1 = p1 + v / 3 # 1/3 point
    q3 = p1 + 2v / 3 # 2/3 point

    # rotated point q2
    θ = -π / 3
    rot60 = @SMatrix [cos(θ) -sin(θ); sin(θ) cos(θ)]
    q2 = q1 + rot60 * (v / 3)
    return [p1, q1, q2, q3, p2]
end

function next_generation(tile::KochTile{2,Float64})
    level = tile.level + 1
    new_vertices = SVector{2,Float64}[]
    old_vertices = tile.vertices
    for i in 1:(length(old_vertices)-1)
        res = subdivide_koch(old_vertices[i], old_vertices[i+1])
        append!(new_vertices, res[1:end-1])
    end
    push!(new_vertices, old_vertices[end])
    center = sum(new_vertices) / length(new_vertices)
    KochTile{2,Float64}(level, new_vertices, center)
end

tile = initiate_koch_tile(level=0)
for g in 1:6
    tile = next_generation(tile)
end
pts = tile.vertices

mat = reduce(hcat, pts)
plot(mat[1, :], mat[2, :], aspect_ratio=1, lw=1, label="Koch Gen 4")


In [ ]:
using StaticArrays, Plots

struct Triangle
    v::SVector{3,SVector{2,Float64}}
end

function subdivide(t::Triangle)
    v1, v2, v3 = t.v
    m12 = (v1 + v2) / 2
    m23 = (v2 + v3) / 2
    m31 = (v3 + v1) / 2

    return (
        Triangle(SVector(v1, m12, m31)),
        Triangle(SVector(v2, m12, m23)),
        Triangle(SVector(v3, m31, m23))
    )
end

function generate_gasket(gen)
    total_tiles = 3^gen
    tiles = Vector{Triangle}(undef, total_tiles)

    # 初期正三角形
    h = sqrt(3) / 2
    tiles[1] = Triangle(SVector(SVector(0.0, 0.0), SVector(1.0, 0.0), SVector(0.5, h)))

    current_count = 1
    for g in 1:gen
        next_tiles = Vector{Triangle}(undef, 3^g)
        for i in 1:current_count
            t1, t2, t3 = subdivide(tiles[i])
            next_tiles[3*(i-1)+1] = t1
            next_tiles[3*(i-1)+2] = t2
            next_tiles[3*(i-1)+3] = t3
        end
        tiles = next_tiles
        current_count = length(tiles)
    end
    return tiles
end
function plot_gasket(tiles)
    p = plot(aspect_ratio=1, legend=false, axis=false, grid=false)
    for t in tiles
        vx = [p[1] for p in t.v]
        vy = [p[2] for p in t.v]
        plot!(p, Shape(vx, vy), color=:purple, linecolor=:white, lw=0.1)
    end
    return p
end

tiles = generate_gasket(6)
plot_gasket(tiles)


In [ ]:
using StaticArrays, Plots

function chaos_game_sierpinski(n_iter::Int)
    vertices = [
        SVector(0.0, 0.0),
        SVector(1.0, 0.0),
        SVector(0.5, sqrt(3) / 2)
    ]
    points = Vector{SVector{2,Float64}}(undef, n_iter)
    current_p = SVector(0, 0)
    for i in 1:n_iter
        target_v = vertices[rand(1:3)]
        current_p = (current_p + target_v) / 2
        points[i] = current_p
    end
    return points
end

n = 10000
pts = chaos_game_sierpinski(n)

# 描画
mat = reduce(hcat, pts)
scatter(mat[1, :], mat[2, :],
    markersize=0.5,
    markerstrokewidth=0,
    color=:blue,
    aspect_ratio=1,
    label="",
    title="Chaos Game: Sierpinski Gasket ($n points)")


In [ ]:
using LinearAlgebra, StaticArrays, Plots

# 再帰的にドラゴン曲線の頂点を計算する
function fill_dragon!(pts, p1, p2, level, sign, offset)
    if level == 0
        pts[offset] = p1
        return offset + 1
    end

    v = p2 - p1
    pmid = p1 + 0.5 * v + sign * 0.5 * SVector(-v[2], v[1])

    mid_idx = offset + 2^(level - 1)
    fill_dragon!(pts, p1, pmid, level - 1, 1, offset)
    fill_dragon!(pts, pmid, p2, level - 1, -1, mid_idx)
end

function generate_dragon(level)
    n_points = 2^level + 1
    pts = Vector{SVector{2,Float64}}(undef, n_points)

    p1 = SVector(0.0, 0.0)
    p2 = SVector(1.0, 0.0)

    fill_dragon!(pts, p1, p2, level, 1, 1)
    pts[end] = p2
    return pts
end

level = 10
@time pts = generate_dragon(level)

mat = reduce(hcat, pts)
plot(mat[1, :], mat[2, :],
    aspect_ratio=1,
    lw=0.5,
    label="Dragon Curve (Gen $level)",
)


In [ ]:
using LinearAlgebra, StaticArrays, Plots

abstract type AbstractTile{N,T} end
struct HeiweiDragon{N,T} <: AbstractTile{N,T}
    level::Int
    vertices::Vector{SVector{N,T}}
    center::SVector{N,T}
end
function initialize_dragon(; level=0)
    v1 = SVector{2,Float64}(0.0, 0.0)
    v2 = SVector{2,Float64}(1.0, 0.0)
    center = (v1 + v2) / 3
    HeiweiDragon{2,Float64}(level, [v1, v2], center)
end
function get_pmid(p1, p2, sign)
    v = p2 - p1
    θ = sign * π / 4
    rot = @SMatrix [cos(θ) -sin(θ); sin(θ) cos(θ)]
    return p1 + (1 / √2) * rot * v
end
function fill_dragon!(pts, p1, p2, level, sign, offset)
    if level == 0
        pts[offset] = p1
        return offset + 1
    end
    pmid = get_pmid(p1, p2, sign)

    step = 2^(level - 1)
    fill_dragon!(pts, p1, pmid, level - 1, 1, offset)
    fill_dragon!(pts, pmid, p2, level - 1, -1, offset + step)
end
function generate_dragon(level)
    n_pts = 2^level + 1
    pts = Vector{SVector{2,Float64}}(undef, n_pts)

    p1 = SVector(0.0, 0.0)
    p2 = SVector(1.0, 0.0)

    fill_dragon!(pts, p1, p2, level, 1, 1)
    pts[end] = p2
    return pts
end
level = 13
pts = generate_dragon(level)
mat = reduce(hcat, pts)

plot(mat[1, :], mat[2, :],
    aspect_ratio=1,
    lw=0.5,
    color=:magma,
    title="Heighway Dragon (Gen $level)",
    label=""
)


In [ ]:
using LinearAlgebra, StaticArrays, Plots
abstract type AbstractTile{N,T} end
struct HeiweiDragon{N,T} <: AbstractTile{N,T}
    level::Int
    vertices::Vector{SVector{N,T}}
    center::SVector{N,T}
end

function get_pmid(p1, p2, sign)
    v = p2 - p1
    θ = sign * π / 4
    rot = @SMatrix [cos(θ) -sin(θ); sin(θ) cos(θ)]
    return p1 + (1 / √2) * rot * v
end
function fill_dragon!(pts, p1, p2, level, sign, offset)
    if level == 0
        pts[offset] = p1
        return offset + 1
    end
    pmid = get_pmid(p1, p2, sign)

    step = 2^(level - 1)
    fill_dragon!(pts, p1, pmid, level - 1, 1, offset)
    fill_dragon!(pts, pmid, p2, level - 1, -1, offset + step)
end

function generate_dragon_tile(level)
    n_pts = 2^level + 1
    pts = Vector{SVector{2,Float64}}(undef, n_pts)

    p1 = SVector(0.0, 0.0)
    p2 = SVector(1.0, 0.0)

    fill_dragon!(pts, p1, p2, level, 1, 1)
    pts[end] = p2

    center = sum(pts) / length(pts)

    return HeiweiDragon{2,Float64}(level, pts, center)
end
level = 13
dragon_tile = generate_dragon_tile(level)

mat = reduce(hcat, dragon_tile.vertices)

plot(mat[1, :], mat[2, :],
    aspect_ratio=1,
    lw=0.5,
    color=:magma,
    title="Heighway Dragon (Gen $(dragon_tile.level))",
    label=""
)


In [ ]:

level = 13
dragon_tile = generate_dragon_tile(level)

pts0 = dragon_tile.vertices
pts1 = rotate_vertices(pts0, π / 2)
pts2 = rotate_vertices(pts0, π)
pts3 = rotate_vertices(pts0, 3π / 2)

p = plot(aspect_ratio=1, title="Heighway Dragon Tiling (4-fold symmetry)")

plot!(p, [v[1] for v in pts0], [v[2] for v in pts0], lw=0.5, alpha=0.80, color=:red, label="Dragon 1")
plot!(p, [v[1] for v in pts1], [v[2] for v in pts1], lw=0.5, alpha=0.80, color=:blue, label="Dragon 2")
plot!(p, [v[1] for v in pts2], [v[2] for v in pts2], lw=0.5, alpha=0.80, color=:green, label="Dragon 3")
plot!(p, [v[1] for v in pts3], [v[2] for v in pts3], lw=0.5, alpha=0.80, color=:orange, label="Dragon 4")

display(p)


In [ ]:
struct FibonacciTile{N,T} <: AbstractTile{N,T}
    level::Int
    str::String
end
function initialize_fibonacci_tile(; level=0)
    FibonacciTile{1,Float64}(level, "S")
end
function grow_fibonacci(tile::FibonacciTile{1,Float64})
    new_str = replace(tile.str, "S" => "L", "L" => "LS")
    new_level = tile.level + 1
    FibonacciTile{1,Float64}(new_level, new_str)
end
tiles = initialize_fibonacci_tile(level=0)
for g in 1:10
    tiles = grow_fibonacci(tiles)
    @show tiles
end


In [ ]:
struct Dragon{N,T} <: AbstractTile{N,T}
    level::Int
    str::String
    position::Vector{SVector{N,T}}
end
function initialize_dragon_tile(; level=0)
    Dragon{2,Float64}(level, "FX", [SVector(0.0, 0.0)])
end
function grow_dragon(tile::Dragon{2,Float64})
    new_str = replace(tile.str, "X" => "X+YF+", "Y" => "-FX-Y")
    new_level = tile.level + 1
    Dragon{2,Float64}(new_level, new_str, tile.position)
end
function string2positions(tile::Dragon{2,Float64})
    pos = tile.position[1]
    direction = SVector(0.0, 1.0)
    positions = SVector{2,Float64}[]
    push!(positions, pos)

    for cmd in collect(tile.str)
        if cmd == 'F'
            pos += direction
            push!(positions, pos)
        elseif cmd == '+'
            θ = π / 2
            rot = @SMatrix [cos(θ) -sin(θ); sin(θ) cos(θ)]
            direction = rot * direction
        elseif cmd == '-'
            θ = -π / 2
            rot = @SMatrix [cos(θ) -sin(θ); sin(θ) cos(θ)]
            direction = rot * direction
        end
    end
    return Dragon{2,Float64}(tile.level, tile.str, positions)
end

tiles = initialize_dragon_tile(level=0)
for g in 1:14
    tiles = grow_dragon(tiles)
end
tile = string2positions(tiles)

mat = reduce(hcat, tile.position)
p = plot(aspect_ratio=1, legend=false, axis=false, grid=false)
plot!(p, mat[1, :], mat[2, :], lw=0.5, color=:blue)


In [ ]:
using LinearAlgebra, StaticArrays, Plots

const ϕ = (1 + √5) / 2
abstract type PenroseTriangle end

struct AcuteTriangle <: PenroseTriangle
    v::SVector{3,SVector{2,Float64}}
end
struct ObtuseTriangle <: PenroseTriangle
    v::SVector{3,SVector{2,Float64}}
end

function subdivide(t::AcuteTriangle)
    v1, v2, v3 = t.v
    p = v1 + (v2 - v1) / ϕ
    return [
        ObtuseTriangle(SVector(v3, p, v2)),
        AcuteTriangle(SVector(p, v3, v1))
    ]
end

function subdivide(t::ObtuseTriangle)
    v1, v2, v3 = t.v
    p1 = v2 + (v1 - v2) / ϕ
    p2 = v2 + (v3 - v2) / ϕ
    return [
        ObtuseTriangle(SVector(p2, p1, v2)),
        ObtuseTriangle(SVector(p1, p2, v3)),
        AcuteTriangle(SVector(p1, v1, v3))
    ]
end

function evolve_penrose(tiles)
    new_tiles = PenroseTriangle[]
    for t in tiles
        append!(new_tiles, subdivide(t))
    end
    return new_tiles
end

function initialize_sun()
    tiles = PenroseTriangle[]
    c = SVector(0.0, 0.0)
    for i in 0:9
        θ1 = i * π / 5
        θ2 = (i + 1) * π / 5
        v1 = c + SVector(cos(θ1), sin(θ1))
        v2 = c + SVector(cos(θ2), sin(θ2))
        # 向きによってAcuteかObtuseか調整
        push!(tiles, AcuteTriangle(SVector(c, v1, v2)))
    end
    return tiles
end


In [ ]:
using LinearAlgebra, StaticArrays, Plots

const ϕ = (1 + √5) / 2
abstract type PenroseTriangle end

struct AcuteTriangle <: PenroseTriangle
    v::SVector{3,SVector{2,Float64}}
end
struct ObtuseTriangle <: PenroseTriangle
    v::SVector{3,SVector{2,Float64}}
end

function subdivide(t::AcuteTriangle)
    v1, v2, v3 = t.v
    p = v1 + (v2 - v1) / ϕ
    return [
        ObtuseTriangle(SVector(v3, p, v2)),
        AcuteTriangle(SVector(p, v3, v1))
    ]
end

function subdivide(t::ObtuseTriangle)
    v1, v2, v3 = t.v
    p1 = v2 + (v1 - v2) / ϕ
    p2 = v2 + (v3 - v2) / ϕ
    return [
        ObtuseTriangle(SVector(p2, p1, v2)),
        ObtuseTriangle(SVector(p1, p2, v3)),
        AcuteTriangle(SVector(p1, v1, v3))
    ]
end

function evolve_penrose(tiles)
    new_tiles = PenroseTriangle[]
    for t in tiles
        append!(new_tiles, subdivide(t))
    end
    return new_tiles
end

function initialize_sun()
    tiles = PenroseTriangle[]
    c = SVector(0.0, 0.0)
    for i in 0:9
        θ1 = i * π / 5
        θ2 = (i + 1) * π / 5
        v1 = c + SVector(cos(θ1), sin(θ1))
        v2 = c + SVector(cos(θ2), sin(θ2))
        # 偶数番目と奇数番目で鏡像になるよう配置
        if i % 2 == 0
            push!(tiles, AcuteTriangle(SVector(c, v2, v1)))
        else
            push!(tiles, AcuteTriangle(SVector(c, v1, v2)))
        end
    end
    return tiles
end
function plot_penrose(tiles)
    p = plot(aspect_ratio=1, legend=false, axis=false, grid=false, ticks=false)
    for t in tiles
        vx = [v[1] for v in t.v]
        vy = [v[2] for v in t.v]
        color = (t isa AcuteTriangle) ? :azure3 : :darkgoldenrod1
        plot!(p, Shape(vx, vy), color=color, linecolor=:white, lw=0.1)
    end
    return p
end

tiles = initialize_sun()
for _ in 1:6
    global tiles = evolve_penrose(tiles)
end
plot_penrose(tiles)
